# Preprocessing data for train set

## to_ann_39 & to_transformer

In [ ]:
import pandas as pd
import ast
import numpy as np

# Đọc CSV
df = pd.read_csv('/mnt/DataSamsung/project/Research_ThyroidFNA_ClassAI/phase3_140924/data/temp/train_not_aug.csv')

# Hàm chuyển predicted_vector từ string thành list
def to_ann_39(vec_str):
    return np.array(ast.literal_eval(vec_str)).flatten()

def to_transformer(vec_str):
    return np.array(ast.literal_eval(vec_str))

# Áp dụng hàm cho từng bản ghi để chuyển đổi predicted_vector từ string thành numpy array
df['to_ann_39'] = df['predicted_vector'].apply(to_ann_39)
df['to_transformer'] = df['predicted_vector'].apply(to_transformer)

# type(df['to_ann_39'][0])      # numpy.ndarray
# df['to_ann_39'][0].shape      # (39,)

# type(df['to_transformer'][0])   # numpy.ndarray
# df['to_transformer'][0].shape   # (13, 3)

## reduce dimension

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Standardize dữ liệu
X = np.vstack(df['to_ann_39'].values)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Sử dụng PCA
pca = PCA()
pca.fit(X_scaled)

# Tính variance explained ratio
explained_variance = pca.explained_variance_ratio_

# Elbow Method
plt.plot(range(1, len(explained_variance)+1), np.cumsum(explained_variance), marker='o', linestyle='--')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA Elbow Method')
plt.show()

In [ ]:
import pickle

# Sử dụng PCA giữ lại 95% thông tin
pca_95 = PCA(0.95)
X_pca_95 = pca_95.fit_transform(X_scaled)

# In ra số chiều sau khi giảm
print(f'Số chiều sau khi giảm: {X_pca_95.shape[1]}')

# Lưu kết quả PCA vào cột mới trong DataFrame
df['pca95_to_ann'] = list(X_pca_95)

# Lưu cấu trúc PCA để áp dụng cho tập val và test
with open('pca_95.pkl', 'wb') as f:
    pickle.dump(pca_95, f)

In [ ]:
# Chọn số chiều phù hợp từ Elbow Method
n_components_optimal = np.argmax(np.cumsum(explained_variance) >= 0.90) + 1  # Ví dụ chọn số chiều khi explained variance đạt 90%

print(f'Số chiều tối ưu: {n_components_optimal}')

# Sử dụng PCA với số chiều từ Elbow Method
pca_elbow = PCA(n_components=n_components_optimal)
X_pca_elbow = pca_elbow.fit_transform(X_scaled)

# Lưu kết quả PCA từ Elbow Method vào cột mới
df['pcaelbow_to_ann'] = list(X_pca_elbow)

# Lưu cấu trúc PCA từ Elbow Method để áp dụng cho tập val và test
with open('pca_elbow.pkl', 'wb') as f:
    pickle.dump(pca_elbow, f)

In [ ]:
from sklearn.cross_decomposition import PLSRegression
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

# Sử dụng PLSRegression để giảm chiều về 2
pls = PLSRegression(n_components=2)
X_pls = pls.fit_transform(X_scaled, df['label'])[0]

# Lưu kết quả PLS vào DataFrame
df['pls_to_ann'] = list(X_pls)

# Trực quan hóa dữ liệu sau khi giảm chiều
# Định nghĩa colormap cho 3 nhãn
cmap = ListedColormap(['r', 'g', 'b'])  # Đỏ cho nhãn 0, xanh lá cho nhãn 1, xanh dương cho nhãn 2

# Trực quan hóa dữ liệu sau khi giảm chiều
plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_pls[:, 0], X_pls[:, 1], c=df['label'], cmap=cmap, edgecolor='k')

# Gắn nhãn cho trục và tiêu đề
plt.xlabel('PLS Component 1')
plt.ylabel('PLS Component 2')
plt.title('PLS Reduced Data Visualization')

# Tạo colorbar với các nhãn
cbar = plt.colorbar(scatter, ticks=[0, 1, 2])
cbar.set_label('Label')
cbar.set_ticks([0, 1, 2])  # Thiết lập chính xác 3 giá trị nhãn
cbar.set_ticklabels(['Class 0', 'Class 1', 'Class 2'])

# Hiển thị biểu đồ
plt.show()

In [ ]:
df.drop(columns=['path_to_image', 'predicted_vector', 'predicted_label'], inplace=True)
df.head()

# Sau khi có thêm PLS, lưu lại DataFrame thành CSV
df.to_csv('train_set_processed_data_with_pca_pls.csv', index=False)

# Preprocessing data for val & test set

In [ ]:
df_val = pd.read_csv('/mnt/DataSamsung/project/Research_ThyroidFNA_ClassAI/phase3_140924/data/temp/val_not_aug.csv')
df_test = pd.read_csv('/mnt/DataSamsung/project/Research_ThyroidFNA_ClassAI/phase3_140924/data/temp/test_not_aug.csv')

...

In [ ]:
# Load lại PCA đã lưu
with open('pca_95.pkl', 'rb') as f:
    pca_95_loaded = pickle.load(f)

with open('pca_elbow.pkl', 'rb') as f:
    pca_elbow_loaded = pickle.load(f)

## More work to do here

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import joblib
import os
import numpy as np

# Đọc dữ liệu từ train set
df_train = pd.read_csv("/mnt/DataSamsung/project/Research_ThyroidFNA_ClassAI/phase3_140924/data/temp/train_not_aug.csv")
df_val = pd.read_csv("/mnt/DataSamsung/project/Research_ThyroidFNA_ClassAI/phase3_140924/data/temp/val_not_aug.csv")
df_test = pd.read_csv("/mnt/DataSamsung/project/Research_ThyroidFNA_ClassAI/phase3_140924/data/temp/test_not_aug.csv")

# Chuyển đổi mỗi chuỗi thành mảng numpy
df_train['predicted_vector'] = df_train['predicted_vector'].apply(lambda x: np.array(eval(x)))
df_val['predicted_vector'] = df_val['predicted_vector'].apply(lambda x: np.array(eval(x)))
df_test['predicted_vector'] = df_test['predicted_vector'].apply(lambda x: np.array(eval(x)))

# Scale dữ liệu trước khi áp dụng PCA/PLS
X_train_scaled = df_train['predicted_vector']
X_val_scaled = df_val['predicted_vector']
X_test_scaled = df_test['predicted_vector']

# PCA giữ 95% thông tin
pca_95 = PCA(0.99)  # n_components=0.95
X_train_pca95 = pca_95.fit_transform(X_train_scaled)
X_val_pca95 = pca_95.transform(X_val_scaled)
X_test_pca95 = pca_95.transform(X_test_scaled)

# Lưu PCA 95 để sử dụng sau này
joblib.dump(pca_95, './pca_95.pkl')

# Lưu lại PCA này và thêm vào df_train, df_val, df_test
df_train['pca95_to_ann'] = X_train_pca95.tolist()
df_val['pca95_to_ann'] = X_val_pca95.tolist()
df_test['pca95_to_ann'] = X_test_pca95.tolist()

# PCA bằng elbow method
pca = PCA()
pca.fit(X_train_scaled)
# Tính variance explained ratio
explained_variance = pca.explained_variance_ratio_
# Elbow Method
plt.plot(range(1, len(explained_variance)+1), np.cumsum(explained_variance), marker='o', linestyle='--')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA Elbow Method')
plt.show()

In [ ]:
pca_elbow = PCA(n_components=5)  # Chọn số chiều từ elbow method đã xác định trước
X_train_pca_elbow = pca_elbow.fit_transform(X_train_scaled)
X_val_pca_elbow = pca_elbow.transform(X_val_scaled)
X_test_pca_elbow = pca_elbow.transform(X_test_scaled)

# Lưu PCA elbow để sử dụng sau này
joblib.dump(pca_elbow, './pca_elbow.pkl')

# Lưu lại PCA này và thêm vào df_train, df_val, df_test
df_train['pcaelbow_to_ann'] = X_train_pca_elbow.tolist()
df_val['pcaelbow_to_ann'] = X_val_pca_elbow.tolist()
df_test['pcaelbow_to_ann'] = X_test_pca_elbow.tolist()

# PLS Regression để giảm về 2 chiều
pls = PLSRegression(n_components=2)
X_train_pls = pls.fit_transform(X_train_scaled, df_train['label'])[0]
X_val_pls = pls.transform(X_val_scaled)
X_test_pls = pls.transform(X_test_scaled)

# Lưu PLS model để sử dụng sau này
joblib.dump(pls, './pls_model.pkl')

# Trực quan hóa tập train sau khi PLS
plt.figure(figsize=(8, 6))
plt.scatter(X_train_pls[:, 0], X_train_pls[:, 1], c=df_train['label'], cmap='viridis', edgecolor='k')
plt.xlabel('PLS Component 1')
plt.ylabel('PLS Component 2')
plt.title('PLS Reduced Data Visualization (Train Set)')
# Thêm thanh màu (colorbar) và đảm bảo rằng chỉ có 3 nhãn (0, 1, 2)
cbar = plt.colorbar(scatter, ticks=[0, 1, 2])
cbar.set_label('Label')
plt.show()

# Lưu lại kết quả PLS vào dataframe
df_train['pls_to_ann'] = X_train_pls.tolist()
df_val['pls_to_ann'] = X_val_pls.tolist()
df_test['pls_to_ann'] = X_test_pls.tolist()

# Lưu file CSV cho train, val và test
df_train.to_csv("./train_set_with_pca_pls.csv", index=False)
df_val.to_csv("./val_set_with_pca_pls.csv", index=False)
df_test.to_csv("./test_set_with_pca_pls.csv", index=False)

# Load lại dữ liệu để chuẩn bị cho deep learning
df_train = pd.read_csv("./train_set_with_pca_pls.csv")
df_val = pd.read_csv("./val_set_with_pca_pls.csv")
df_test = pd.read_csv("./test_set_with_pca_pls.csv")

# Lựa chọn các trường để train mạng deep learning
X_train = df_train['pca95_to_ann'].apply(eval).tolist()
y_train = df_train['label'].values

X_val = df_val['pca95_to_ann'].apply(eval).tolist()
y_val = df_val['label'].values

X_test = df_test['pca95_to_ann'].apply(eval).tolist()
y_test = df_test['label'].values

# Bây giờ bạn có thể sử dụng X_train, y_train, X_val, y_val, X_test, y_test để train mô hình deep learning

# Load lại mô hình PCA/PLS đã lưu để sử dụng tiếp
scaler = joblib.load('./scaler.pkl')
pca_95 = joblib.load('./pca_95.pkl')
pca_elbow = joblib.load('./pca_elbow.pkl')
pls = joblib.load('./pls_model.pkl')